Reference: https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0?gi=b3b53e8290cd

In [1]:
import warnings
import gensim
import spacy
import joblib
import pickle
import pyLDAvis
import tqdm
import pandas as pd
import numpy as np
import pyLDAvis.gensim_models as gensimvis
import gensim.corpora as corpora

from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from datetime import datetime

warnings.filterwarnings(action="ignore")
pyLDAvis.enable_notebook()

/Users/micocomia/.pyenv/versions/3.9.1/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


### I. Import Data
---

In [2]:
gcash_good = joblib.load('../../Data/Preprocessed/Mico/P2/gcash_good.sav')
gcash_bad = joblib.load('../../Data/Preprocessed/Mico/P2/gcash_bad.sav')
paymaya_good = joblib.load('../../Data/Preprocessed/Mico/P2/paymaya_good.sav')
paymaya_bad = joblib.load('../../Data/Preprocessed/Mico/P2/paymaya_bad.sav')

In [3]:
def get_data(list, store):
    if store == 'as':
        data, dictionary, corpus = list[0], list[1], list[2]
    elif store == 'ps':
        data, dictionary, corpus = list[3], list[4], list[5]
    return data, dictionary, corpus

In [4]:
gcash_good_as, gcash_good_dict_as, gcash_good_corpus_as = get_data(gcash_good, 'as')
gcash_good_ps, gcash_good_dict_ps, gcash_good_corpus_ps = get_data(gcash_good, 'ps')

In [5]:
gcash_bad_as, gcash_bad_dict_as, gcash_bad_corpus_as = get_data(gcash_bad, 'as')
gcash_bad_ps, gcash_bad_dict_ps, gcash_bad_corpus_ps = get_data(gcash_bad, 'ps')

In [6]:
paymaya_good_as, paymaya_good_dict_as, paymaya_good_corpus_as = get_data(paymaya_good, 'as')
paymaya_good_ps, paymaya_good_dict_ps, paymaya_good_corpus_ps = get_data(paymaya_good, 'ps')

In [7]:
paymaya_bad_as, paymaya_bad_dict_as, paymaya_bad_corpus_as = get_data(paymaya_bad, 'as')
paymaya_bad_ps, paymaya_bad_dict_ps, paymaya_bad_corpus_ps = get_data(paymaya_bad, 'ps')

### II. Topic Modeling
---

#### A. Base Model

In [8]:
def build_model(corpus, dictionary):
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=10000, # Increasing chunksize will speed up training as long as document fits memory
                                           passes=10, # Epochs
                                           per_word_topics=True)
    return lda_model

Create different LDA models for each type of review and store.

In [9]:
gcash_good_lda_as = build_model(gcash_good_corpus_as, gcash_good_dict_as)
gcash_bad_lda_as = build_model(gcash_bad_corpus_as, gcash_bad_dict_as)

In [10]:
gcash_good_lda_ps = build_model(gcash_good_corpus_ps, gcash_good_dict_ps)
gcash_bad_lda_ps = build_model(gcash_bad_corpus_ps, gcash_bad_dict_ps)

In [11]:
paymaya_good_lda_as = build_model(paymaya_good_corpus_as, paymaya_good_dict_as)
paymaya_bad_lda_as = build_model(paymaya_bad_corpus_as, paymaya_bad_dict_as)

In [12]:
paymaya_good_lda_ps = build_model(paymaya_good_corpus_ps, paymaya_good_dict_ps)
paymaya_bad_lda_ps = build_model(paymaya_bad_corpus_ps, paymaya_bad_dict_ps)

#### B. Base Coherence

Check quality of topics through topic coherence which is measured by the "degree of semantic similarity between hgih scoring words in the topics." We first compute the base coherence of the topics.

In [13]:
def compute_coherence(lda_model, data, dictionary):
    coherence_model_lda = CoherenceModel(model = lda_model,
                                         texts = data,
                                         dictionary = dictionary, 
                                         coherence = 'c_v')
    
    return coherence_model_lda.get_coherence()

In [14]:
gcash_good_as_base = compute_coherence(gcash_good_lda_as, gcash_good_as, gcash_good_dict_as)
gcash_bad_as_base = compute_coherence(gcash_bad_lda_as, gcash_bad_as, gcash_bad_dict_as)
gcash_good_ps_base = compute_coherence(gcash_good_lda_ps, gcash_good_ps, gcash_good_dict_ps)
gcash_bad_ps_base = compute_coherence(gcash_bad_lda_ps, gcash_bad_ps, gcash_bad_dict_ps)

In [15]:
paymaya_good_as_base = compute_coherence(paymaya_good_lda_as, paymaya_good_as, paymaya_good_dict_as)
paymaya_bad_as_base = compute_coherence(paymaya_bad_lda_as, paymaya_bad_as, paymaya_bad_dict_as)
paymaya_good_ps_base = compute_coherence(paymaya_good_lda_ps, paymaya_good_ps, paymaya_good_dict_ps)
paymaya_bad_ps_base = compute_coherence(paymaya_bad_lda_ps, paymaya_bad_ps, paymaya_bad_dict_ps)

In [16]:
gcash_as = [gcash_good_as_base, gcash_bad_as_base]
gcash_ps = [gcash_good_ps_base, gcash_bad_ps_base]
paymaya_as = [paymaya_good_as_base, paymaya_bad_as_base]
paymaya_ps = [paymaya_good_ps_base, paymaya_bad_ps_base]

In [17]:
df = pd.DataFrame(list(zip(gcash_as, gcash_ps, paymaya_as, paymaya_ps)), 
                  columns = ['GCash AS', 'GCash PS', 'PayMaya AS', 'PayMaya PS'],
                  index = ['Good', 'Bad'])

In [18]:
df

,GCash AS,GCash PS,PayMaya AS,PayMaya PS
Good,0.429972,0.461109,0.295934,0.549764
Bad,0.391118,0.422193,0.299370,0.463847


In [19]:
df.to_csv('Results/base.csv', index=False)

#### C. Hyperparameter Tuning

In [8]:
# supporting function
def compute_coherence_values(corpus, dictionary, data, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k,
                                           random_state=100,
                                           chunksize=10000,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [9]:
def hyperparameter_tuner(corpus, dictionary, data, temp):
    grid = {}
    grid['Validation_Set'] = {}

    # Topics range
    min_topics = 2
    max_topics = 6
    step_size = 1
    topics_range = range(min_topics, max_topics, step_size)

    # Alpha parameter
    alpha = list(np.arange(0.01, 1, 0.3))
    alpha.append('symmetric')
    alpha.append('asymmetric')

    # Beta parameter
    beta = list(np.arange(0.01, 1, 0.3))
    beta.append('symmetric')

    # Validation sets
    num_of_docs = len(corpus)
    corpus_sets = [corpus]
    corpus_title = ['100% Corpus']
    model_results = {'Validation_Set': [],
                     'Topics': [],
                     'Alpha': [],
                     'Beta': [],
                     'Coherence': []
                    }

    # Can take a long time to run
    if 1 == 1:
        pbar = tqdm.tqdm(total=120)

        # iterate through validation corpuses
        for i in range(len(corpus_sets)):
            # iterate through number of topics
            for k in topics_range:
                # iterate through alpha values
                for a in alpha:
                    # iterare through beta values
                    for b in beta:
                        # get the coherence score for the given parameters
                        cv = compute_coherence_values(corpus_sets[i], 
                                                      dictionary, 
                                                      data,
                                                      k, a, b)
                        # Save the model results
                        model_results['Validation_Set'].append(corpus_title[i])
                        model_results['Topics'].append(k)
                        model_results['Alpha'].append(a)
                        model_results['Beta'].append(b)
                        model_results['Coherence'].append(cv)

                        pbar.update(1)
        filename = f'Results/{temp}.csv'
        pd.DataFrame(model_results).to_csv(filename, index=False)
        pbar.close()

In [10]:
hyperparameter_tuner(gcash_good_corpus_as, gcash_good_dict_as, gcash_good_as, 'gcash_good_as')

100%|██████████| 120/120 [06:06<00:00,  3.05s/it]


In [11]:
hyperparameter_tuner(gcash_bad_corpus_as, gcash_bad_dict_as, gcash_bad_as, 'gcash_bad_as')

100%|██████████| 120/120 [18:18<00:00,  9.16s/it]


In [12]:
hyperparameter_tuner(gcash_good_corpus_ps, gcash_good_dict_ps, gcash_good_ps, 'gcash_good_ps')

100%|██████████| 120/120 [56:47<00:00, 28.40s/it]


In [13]:
hyperparameter_tuner(gcash_bad_corpus_ps, gcash_bad_dict_ps, gcash_bad_ps, 'gcash_bad_ps')

100%|██████████| 120/120 [1:25:25<00:00, 42.71s/it]


In [14]:
hyperparameter_tuner(paymaya_good_corpus_as, paymaya_good_dict_as, paymaya_good_as, 'paymaya_good_as')

 17%|█▋        | 20/120 [01:15<06:23,  3.83s/it]

KeyboardInterrupt: 

In [ ]:
hyperparameter_tuner(paymaya_bad_corpus_as, paymaya_bad_dict_as, paymaya_bad_as, 'paymaya_bad_as')

In [ ]:
hyperparameter_tuner(paymaya_good_corpus_ps, paymaya_good_dict_ps, paymaya_good_ps, 'paymaya_good_ps')

In [ ]:
hyperparameter_tuner(paymaya_bad_corpus_ps, paymaya_bad_dict_ps, paymaya_bad_ps, 'paymaya_bad_ps')

#### D. Final Models

In [15]:
def build_final_model(df, corpus, dictionary):
    df1 = df.iloc[df['Coherence'].idxmax]
    alpha = df1.Alpha
    beta = df1.Beta
    topics = df1.Topics
    
    if (alpha != 'symmetric') and (alpha != "asymmetric"):
        alpha = float(alpha)
        
    if (beta != 'symmetric'):
        beta = float(beta)
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=topics, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=alpha,
                                           eta=beta)
    
    return lda_model

In [16]:
df_gcash_good_as = pd.read_csv('Results/gcash_good_as.csv')
df_gcash_bad_as = pd.read_csv('Results/gcash_bad_as.csv')
df_gcash_good_ps = pd.read_csv('Results/gcash_good_ps.csv')
df_gcash_bad_ps = pd.read_csv('Results/gcash_bad_ps.csv')

In [576]:
df_paymaya_good_as = pd.read_csv('Results/paymaya_good_as.csv')
df_paymaya_bad_as = pd.read_csv('Results/paymaya_bad_as.csv')
df_paymaya_good_ps = pd.read_csv('Results/paymaya_good_ps.csv')
df_paymaya_bad_ps = pd.read_csv('Results/paymaya_bad_ps.csv')

In [17]:
gcash_good_as_final = build_final_model(df_gcash_good_as, gcash_good_corpus_as, gcash_good_dict_as)
gcash_bad_as_final = build_final_model(df_gcash_bad_as, gcash_bad_corpus_as, gcash_bad_dict_as)
gcash_good_ps_final = build_final_model(df_gcash_good_ps, gcash_good_corpus_ps, gcash_good_dict_ps)
gcash_bad_ps_final = build_final_model(df_gcash_bad_ps, gcash_bad_corpus_ps, gcash_bad_dict_ps)

In [578]:
paymaya_good_as_final = build_final_model(df_paymaya_good_as, paymaya_good_corpus_as, paymaya_good_dict_as)
paymaya_bad_as_final = build_final_model(df_paymaya_bad_as, paymaya_bad_corpus_as, paymaya_bad_dict_as)
paymaya_good_ps_final = build_final_model(df_paymaya_good_ps, paymaya_good_corpus_ps, paymaya_good_dict_ps)
paymaya_bad_ps_final = build_final_model(df_paymaya_bad_ps, paymaya_bad_corpus_ps, paymaya_bad_dict_ps)

### III. Topic Visualization 
---

In [18]:
gcash_good_as_visual = gensimvis.prepare(gcash_good_as_final, gcash_good_corpus_as, gcash_good_dict_as)
print(gcash_good_as_final.print_topics())
gcash_good_as_visual

[(0, '0.156*"money" + 0.097*"pay" + 0.080*"bills" + 0.078*"transactions" + 0.068*"bank" + 0.068*"cash" + 0.061*"online" + 0.060*"transfer" + 0.060*"use" + 0.049*"hope"'), (1, '0.149*"good" + 0.143*"love" + 0.125*"convenient" + 0.095*"use" + 0.078*"need" + 0.074*"great" + 0.060*"transaction" + 0.059*"time" + 0.049*"useful" + 0.049*"easy"')]


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.122718  0.0       1        1  57.201171
1     -0.122718  0.0       2        1  42.798829, topic_info=            Term       Freq      Total Category  logprob  loglift
1           good  31.000000  31.000000  Default  19.0000  19.0000
8           love  31.000000  31.000000  Default  18.0000  18.0000
4          money  42.000000  42.000000  Default  17.0000  17.0000
12           pay  25.000000  25.000000  Default  16.0000  16.0000
3          bills  21.000000  21.000000  Default  15.0000  15.0000
7           need  19.000000  19.000000  Default  14.0000  14.0000
0     convenient  36.000000  36.000000  Default  13.0000  13.0000
11        online  16.000000  16.000000  Default  12.0000  12.0000
13      transfer  16.000000  16.000000  Default  11.0000  11.0000
15         great  19.000000  19.000000  Default  10.0000  10.0000
18   transaction  16.000000  16.000000  Default   9.0000   9.0000
16          cash  20.000000  20.000000  Default   8.0000   8.0000
9   transactions  24.000000  24.000000  Default   7.0000   7.0000
6            use  33.000000  33.000000  Default   6.0000   6.0000
5           time  19.000000  19.000000  Default   5.0000   5.0000
17        useful  16.000000  16.000000  Default   4.0000   4.0000
10          bank  23.000000  23.000000  Default   3.0000   3.0000
14          easy  19.000000  19.000000  Default   2.0000   2.0000
2           hope  18.000000  18.000000  Default   1.0000   1.0000
12           pay  24.808832  25.131063   Topic1  -2.3372   0.5457
3          bills  20.454220  21.199799   Topic1  -2.5302   0.5228
11        online  15.675275  16.296205   Topic1  -2.7963   0.5197
13      transfer  15.463587  16.292862   Topic1  -2.8099   0.5064
4          money  40.092066  42.753705   Topic1  -1.8572   0.4943
16          cash  17.451973  20.186795   Topic1  -2.6890   0.4130
9   transactions  20.076032  24.090762   Topic1  -2.5489   0.3763
10          bank  17.571343  23.085617   Topic1  -2.6822   0.2857
2           hope  12.621827  18.179327   Topic1  -3.0130   0.1937
14          easy   9.774213  19.100073   Topic1  -3.2687  -0.1113
6            use  15.461906  33.674466   Topic1  -2.8100  -0.2198
17        useful   6.688559  16.154474   Topic1  -3.6480  -0.3232
5           time   7.691517  19.067231   Topic1  -3.5083  -0.3493
0     convenient  12.587796  36.526080   Topic1  -3.0157  -0.5067
18   transaction   4.520203  16.120281   Topic1  -4.0399  -0.7129
15         great   4.855797  19.022513   Topic1  -3.9683  -0.8069
7           need   4.045561  19.009743   Topic1  -4.1508  -0.9887
8           love   4.049949  31.563214   Topic1  -4.1497  -1.4947
1           good   2.942571  31.545758   Topic1  -4.4691  -1.8136
1           good  28.603187  31.545758   Topic2  -1.9048   0.7507
8           love  27.513264  31.563214   Topic2  -1.9437   0.7113
7           need  14.964182  19.009743   Topic2  -2.5527   0.6094
15         great  14.166717  19.022513   Topic2  -2.6075   0.5539
18   transaction  11.600078  16.120281   Topic2  -2.8074   0.5196
0     convenient  23.938284  36.526080   Topic2  -2.0829   0.4261
5           time  11.375714  19.067231   Topic2  -2.8269   0.3322
17        useful   9.465915  16.154474   Topic2  -3.0107   0.3142
6            use  18.212561  33.674466   Topic2  -2.3563   0.2340
14          easy   9.325860  19.100073   Topic2  -3.0256   0.1318
2           hope   5.557500  18.179327   Topic2  -3.5432  -0.3365
10          bank   5.514274  23.085617   Topic2  -3.5510  -0.5832
9   transactions   4.014730  24.090762   Topic2  -3.8684  -0.9432
16          cash   2.734821  20.186795   Topic2  -4.2523  -1.1503
4          money   2.661638  42.753705   Topic2  -4.2794  -1.9279
13      transfer   0.829275  16.292862   Topic2  -5.4456  -2.1293
11        online   0.620930  16.296205   Topic2  -5.7349  -2.4188
3          bills   0.745579  21.199799   Topic2  -5.5520  -2.4989

In [19]:
gcash_bad_as_visual = gensimvis.prepare(gcash_bad_as_final, gcash_bad_corpus_as, gcash_bad_dict_as)
print(gcash_bad_as_final.print_topics())
gcash_bad_as_visual

[(0, '0.042*"money" + 0.025*"load" + 0.019*"verify" + 0.015*"tried" + 0.015*"already" + 0.015*"fix" + 0.015*"use" + 0.013*"get" + 0.013*"mpin" + 0.011*"transaction"'), (1, '0.041*"money" + 0.028*"service" + 0.024*"use" + 0.022*"bank" + 0.018*"customer_service" + 0.014*"working" + 0.014*"transaction" + 0.012*"one" + 0.012*"log" + 0.011*"ticket"'), (2, '0.024*"fix" + 0.021*"verification" + 0.020*"verified" + 0.018*"need" + 0.016*"gcredit" + 0.015*"verify" + 0.014*"use" + 0.013*"problem" + 0.013*"issue" + 0.012*"already"'), (3, '0.048*"cash" + 0.048*"update" + 0.024*"always" + 0.019*"pera" + 0.019*"use" + 0.019*"fix" + 0.016*"wala" + 0.015*"email" + 0.014*"error" + 0.013*"need"')]


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.072212 -0.013443       1        1  28.229678
0      0.037543  0.042062       2        1  25.996841
3     -0.037192 -0.075909       3        1  23.581636
2     -0.072563  0.047291       4        1  22.191845, topic_info=       Term        Freq       Total Category  logprob  loglift
83   update  198.000000  198.000000  Default  30.0000  30.0000
28     cash  218.000000  218.000000  Default  29.0000  29.0000
12    money  325.000000  325.000000  Default  28.0000  28.0000
174    load  118.000000  118.000000  Default  27.0000  27.0000
198    pera   77.000000   77.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
22      use   40.488221  234.313774   Topic4  -4.2649  -0.2502
16   system   23.687692   72.076216   Topic4  -4.8010   0.3927
138     get   25.138375  110.165369   Topic4  -4.7416   0.0279
54     time   25.133859  121.959713   Topic4  -4.7417  -0.0740
28     cash   19.866057  218.598195   Topic4  -4.9769  -0.8928

[227 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
180       1  0.029245             acc
180       2  0.526417             acc
180       3  0.116982             acc
180       4  0.321699             acc
134       1  0.279556          access
...     ...       ...             ...
157       4  0.045503  worst_customer
91        1  0.079545           wrong
91        2  0.258523           wrong
91        3  0.218750           wrong
91        4  0.437500           wrong

[628 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 3])

In [20]:
gcash_good_ps_visual = gensimvis.prepare(gcash_good_ps_final, gcash_good_corpus_ps, gcash_good_dict_ps)
print(gcash_good_ps_final.print_topics())
gcash_good_ps_visual

[(0, '0.258*"good" + 0.166*"nice" + 0.080*"great" + 0.064*"apps" + 0.049*"ok" + 0.048*"useful" + 0.042*"excellent" + 0.030*"helpful" + 0.026*"awesome" + 0.023*"amazing"'), (1, '0.098*"easy" + 0.078*"use" + 0.063*"convenient" + 0.052*"money" + 0.028*"fast" + 0.025*"transaction" + 0.017*"send" + 0.015*"cash" + 0.014*"pay_bills" + 0.014*"payment"'), (2, '0.097*"love" + 0.018*"update" + 0.017*"load" + 0.016*"much" + 0.013*"maganda" + 0.013*"give" + 0.012*"cash" + 0.011*"really" + 0.011*"hope" + 0.009*"need"')]


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.389744  0.006616       1        1  40.935950
1      0.187675 -0.272016       2        1  35.036254
2      0.202069  0.265400       3        1  24.027796, topic_info=       Term          Freq         Total Category  logprob  loglift
6      good  14520.000000  14520.000000  Default  30.0000  30.0000
1      nice   9332.000000   9332.000000  Default  29.0000  29.0000
10     easy   4701.000000   4701.000000  Default  28.0000  28.0000
16     love   3195.000000   3195.000000  Default  27.0000  27.0000
0     great   4508.000000   4508.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
30   really    361.956944    867.049248   Topic3  -4.5121   0.5524
22     cash    409.499914   1140.251801   Topic3  -4.3887   0.4019
24     need    306.392804    730.130611   Topic3  -4.6787   0.5576
12   thanks    226.492062    873.009998   Topic3  -4.9809   0.0767
219    want    175.087545    285.314778   Topic3  -5.2383   0.9377

[160 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
319       1  0.007479      add
319       2  0.014959      add
319       3  0.979810      add
577       1  0.008030  already
577       2  0.008030  already
...     ...       ...      ...
51        2  0.995384      way
51        3  0.002626      way
84        1  0.997935      wow
84        2  0.001577      wow
84        3  0.001577      wow

[369 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

In [21]:
gcash_bad_ps_visual = gensimvis.prepare(gcash_bad_ps_final, gcash_bad_corpus_ps, gcash_bad_dict_ps)
print(gcash_bad_ps_final.print_topics())
gcash_bad_ps_visual

[(0, '0.053*"update" + 0.044*"always" + 0.025*"use" + 0.025*"need" + 0.022*"open" + 0.019*"fix" + 0.017*"good" + 0.014*"error" + 0.014*"time" + 0.013*"log"'), (1, '0.023*"email" + 0.023*"code" + 0.018*"number" + 0.016*"send" + 0.013*"register" + 0.013*"already" + 0.012*"worst" + 0.012*"customer_service" + 0.011*"get" + 0.011*"mpin"'), (2, '0.066*"money" + 0.030*"cash" + 0.017*"load" + 0.015*"transaction" + 0.015*"service" + 0.013*"back" + 0.013*"send" + 0.011*"bank" + 0.009*"pesos" + 0.009*"give"'), (3, '0.028*"pera" + 0.028*"update" + 0.027*"wala" + 0.025*"load" + 0.013*"cash" + 0.012*"ayaw" + 0.009*"ok" + 0.008*"bulok" + 0.008*"tagal" + 0.008*"laging"'), (4, '0.077*"verify" + 0.052*"verified" + 0.051*"id" + 0.031*"student_id" + 0.030*"verification" + 0.022*"fully_verified" + 0.019*"get" + 0.015*"option" + 0.014*"student" + 0.013*"picture"')]


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.033607  0.006300       1        1  30.957899
2     -0.048283 -0.168513       2        1  23.531366
1     -0.129182 -0.131808       3        1  19.627433
3      0.375392  0.043286       4        1  14.360511
4     -0.164321  0.250735       5        1  11.522791, topic_info=         Term         Freq        Total Category  logprob  loglift
72      money  3097.000000  3097.000000  Default  30.0000  30.0000
162    verify  1761.000000  1761.000000  Default  29.0000  29.0000
161    update  3962.000000  3962.000000  Default  28.0000  28.0000
9      always  2776.000000  2776.000000  Default  27.0000  27.0000
53   verified  1176.000000  1176.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
58        get   431.667832  1342.992369   Topic5  -3.9557   1.0258
246     tried   203.734981   923.793795   Topic5  -4.7065   0.6492
19        use   263.068455  2300.363911   Topic5  -4.4509  -0.0076
25        fix   241.448808  1778.108379   Topic5  -4.5367   0.1642
36       says   161.310426   924.078860   Topic5  -4.9400   0.4154

[295 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
809       1  0.006429  accept
809       2  0.006429  accept
809       3  0.006429  accept
809       4  0.006429  accept
809       5  0.983564  accept
...     ...       ...     ...
98        1  0.011360   worst
98        2  0.094668   worst
98        3  0.889882   worst
98        4  0.001893   worst
98        5  0.001893   worst

[1220 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4, 5])

In [583]:
paymaya_good_as_visual = gensimvis.prepare(paymaya_good_as_final, paymaya_good_corpus_as, paymaya_good_dict_as)
print(paymaya_good_as_final.print_topics())
paymaya_good_as_visual

[(0, '0.014*"use" + 0.012*"convenient" + 0.010*"easy" + 0.009*"pay" + 0.007*"great" + 0.007*"good" + 0.007*"money" + 0.007*"need" + 0.007*"really" + 0.006*"love"'), (1, '0.003*"applications" + 0.003*"one" + 0.002*"money" + 0.002*"wallet" + 0.002*"service" + 0.002*"cash" + 0.002*"pay" + 0.002*"never" + 0.002*"email" + 0.002*"tumawag"'), (2, '0.003*"email" + 0.003*"password" + 0.003*"face" + 0.002*"new" + 0.002*"started" + 0.002*"frustrating" + 0.002*"recovery" + 0.002*"since" + 0.002*"sales" + 0.002*"chat"')]


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.067081 -0.000520       1        1  82.539042
1     -0.035102 -0.016494       2        1  12.009478
2     -0.031979  0.017014       3        1   5.451480, topic_info=          Term       Freq      Total Category  logprob  loglift
2        email   5.000000   5.000000  Default  30.0000  30.0000
68       never   9.000000   9.000000  Default  29.0000  29.0000
276        new   3.000000   3.000000  Default  28.0000  28.0000
53       deals   5.000000   5.000000  Default  27.0000  27.0000
38       since  19.000000  19.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
38       since   0.580867  19.720295   Topic3  -6.3629  -0.6156
726   pandemic   0.451876  10.573476   Topic3  -6.6140  -0.2434
587      years   0.427954   8.523415   Topic3  -6.6684  -0.0823
56        ever   0.430857   9.948682   Topic3  -6.6616  -0.2301
722  different   0.410611   8.603316   Topic3  -6.7097  -0.1330

[183 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
356       1  0.481130       address
474       1  0.613895          alam
474       2  0.613895          alam
689       1  0.516957  applications
689       2  0.516957  applications
...     ...       ...           ...
701       1  0.564982        wallet
701       2  0.376655        wallet
997       1  0.826376          wear
553       1  0.826401        wonder
587       1  0.938591         years

[237 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

In [584]:
paymaya_bad_as_visual = gensimvis.prepare(paymaya_bad_as_final, paymaya_bad_corpus_as, paymaya_bad_dict_as)
print(paymaya_bad_as_final.print_topics())
paymaya_bad_as_visual

[(0, '0.015*"money" + 0.012*"upgrade" + 0.009*"use" + 0.008*"tried" + 0.007*"customer_service" + 0.007*"error" + 0.006*"since" + 0.006*"service" + 0.005*"email" + 0.005*"card"'), (1, '0.008*"money" + 0.004*"get" + 0.004*"customer_service" + 0.004*"service" + 0.004*"card" + 0.003*"use" + 0.003*"balance" + 0.003*"fee" + 0.003*"online" + 0.003*"virtual_card"'), (2, '0.003*"replika" + 0.002*"get" + 0.002*"credentials" + 0.002*"worse" + 0.002*"greedy" + 0.002*"us" + 0.002*"users" + 0.001*"feature" + 0.001*"sorry" + 0.001*"old"'), (3, '0.003*"service" + 0.002*"number" + 0.002*"load" + 0.002*"tsk" + 0.002*"poor" + 0.002*"error" + 0.002*"grabe" + 0.002*"reply" + 0.002*"ayos" + 0.002*"wala"'), (4, '0.002*"name" + 0.002*"passport" + 0.001*"valid" + 0.001*"buffer" + 0.001*"match" + 0.001*"gateway" + 0.001*"needs" + 0.001*"screen" + 0.001*"attempt" + 0.001*"hope"')]


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.099401 -0.007465       1        1  77.180667
1     -0.004369  0.026190       2        1  12.948989
2      0.032431 -0.008439       3        1   3.659002
3      0.033422 -0.008026       4        1   3.444717
4      0.037916 -0.002259       5        1   2.766626, topic_info=        Term        Freq       Total Category  logprob  loglift
63     money  146.000000  146.000000  Default  30.0000  30.0000
17       get   25.000000   25.000000  Default  29.0000  29.0000
214  service   59.000000   59.000000  Default  28.0000  28.0000
187   number   40.000000   40.000000  Default  27.0000  27.0000
5       load   45.000000   45.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
750      mas    0.314236    4.544125   Topic5  -6.9486   0.9161
445     hope    0.393054   13.509457   Topic5  -6.7248   0.0503
796    least    0.297326    5.557225   Topic5  -7.0039   0.6595
494   paying    0.290905    7.378237   Topic5  -7.0258   0.3542
108    think    0.287986    7.344824   Topic5  -7.0359   0.3487

[302 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
691       1  0.776591      absolute
232       1  0.955210     add_money
232       2  0.045486     add_money
118       1  0.979135       address
2291      1  0.633599  advancements
...     ...       ...           ...
1487      1  0.670505         worse
1487      3  0.223502         worse
111       1  0.945132         worst
111       2  0.054008         worst
2502      1  0.553143         yadda

[403 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5])

In [585]:
paymaya_good_ps_visual = gensimvis.prepare(paymaya_good_ps_final, paymaya_good_corpus_ps, paymaya_good_dict_ps)
print(paymaya_good_ps_final.print_topics())
paymaya_good_ps_visual

[(0, '0.114*"good" + 0.082*"nice" + 0.059*"great" + 0.049*"convenient" + 0.048*"useful" + 0.044*"apps" + 0.031*"love" + 0.029*"awesome" + 0.023*"use" + 0.023*"helpful"'), (1, '0.043*"easy" + 0.041*"use" + 0.023*"pay" + 0.019*"fast" + 0.018*"money" + 0.017*"transaction" + 0.016*"bills" + 0.015*"online" + 0.011*"service" + 0.009*"transactions"'), (2, '0.019*"really" + 0.016*"need" + 0.011*"one" + 0.011*"online" + 0.010*"time" + 0.009*"lot" + 0.009*"since" + 0.009*"ewallet" + 0.008*"load" + 0.008*"bills"'), (3, '0.008*"sobrang" + 0.008*"gamitin" + 0.007*"maganda" + 0.007*"pera" + 0.007*"wala" + 0.006*"mabilis" + 0.006*"bills" + 0.006*"madali" + 0.006*"pwede" + 0.005*"pang"')]


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.077252 -0.132623       1        1  35.575981
1     -0.018948 -0.185914       2        1  29.031299
0     -0.294838  0.129141       3        1  19.338868
3      0.236534  0.189396       4        1  16.053852, topic_info=            Term         Freq        Total Category  logprob  loglift
0           good  5155.000000  5155.000000  Default  30.0000  30.0000
62          nice  3652.000000  3652.000000  Default  29.0000  29.0000
2          great  2632.000000  2632.000000  Default  28.0000  28.0000
37        useful  2165.000000  2165.000000  Default  27.0000  27.0000
1     convenient  2512.000000  2512.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
1116         mas   151.603071   356.217012   Topic4  -5.5024   0.9749
2385        pala   101.804876   141.606244   Topic4  -5.9006   1.4992
120        bills   218.257544  1973.931053   Topic4  -5.1380  -0.3729
921     services   126.278917   331.406119   Topic4  -5.6852   0.8644
1666     feature    96.420333   427.667910   Topic4  -5.9549   0.3396

[268 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
251       1  0.991748  additional
1512      4  0.992173        agad
4697      4  0.987651       akala
496       2  0.991208      almost
94        1  0.001299     amazing
...     ...       ...         ...
950       1  0.012306   wonderful
950       3  0.910617   wonderful
950       4  0.073834   wonderful
676       2  0.004036         wow
676       3  0.992908         wow

[511 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 4])

In [586]:
paymaya_bad_ps_visual = gensimvis.prepare(paymaya_bad_ps_final, paymaya_bad_corpus_ps, paymaya_bad_dict_ps)
print(paymaya_bad_ps_final.print_topics())
paymaya_bad_ps_visual

[(0, '0.015*"money" + 0.015*"upgrade" + 0.012*"use" + 0.009*"customer_service" + 0.007*"email" + 0.007*"error" + 0.007*"get" + 0.007*"tried" + 0.006*"already" + 0.006*"service"'), (1, '0.007*"upgrade" + 0.007*"id" + 0.005*"student_id" + 0.003*"secondary_id" + 0.003*"primary_id" + 0.003*"list" + 0.002*"sss" + 0.002*"voters_id" + 0.002*"philhealth" + 0.002*"verify"'), (2, '0.012*"wala" + 0.011*"pera" + 0.008*"load" + 0.006*"upgrade" + 0.005*"hirap" + 0.005*"ayaw" + 0.004*"id" + 0.003*"apps" + 0.003*"ok" + 0.003*"error"')]


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.182742 -0.015797       1        1  76.284280
2     -0.128846 -0.049875       2        1  16.748017
1     -0.053896  0.065672       3        1   6.967704, topic_info=           Term         Freq        Total Category  logprob  loglift
397        wala   212.000000   212.000000  Default  30.0000  30.0000
83         pera   199.000000   199.000000  Default  29.0000  29.0000
492          id   334.000000   334.000000  Default  28.0000  28.0000
18      upgrade  1263.000000  1263.000000  Default  27.0000  27.0000
9          load   569.000000   569.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
959         add     9.278252    90.826279   Topic3  -6.6178   0.3826
1200  available     8.079069    67.513723   Topic3  -6.7562   0.5408
136        need     8.965335   477.142352   Topic3  -6.6521  -1.3106
121       money     9.054926  1190.652427   Topic3  -6.6422  -2.2151
284         pay     7.903190   217.087512   Topic3  -6.7782  -0.6491

[191 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
818       1  0.694874     acc
818       2  0.170871     acc
818       3  0.125305     acc
337       1  0.989388  access
337       2  0.006552  access
...     ...       ...     ...
3160      1  0.053581    yong
3160      2  0.910882    yong
1003      1  0.144276     yrs
1003      2  0.144276     yrs
1003      3  0.721382     yrs

[439 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

In [587]:
collated = []
collated.append(str(gcash_good_as_final.print_topics()))
collated.append(str(gcash_bad_as_final.print_topics()))
collated.append(str(gcash_good_ps_final.print_topics()))
collated.append(str(gcash_bad_ps_final.print_topics()))
collated.append(str(paymaya_good_as_final.print_topics()))
collated.append(str(paymaya_bad_as_final.print_topics()))
collated.append(str(paymaya_good_ps_final.print_topics()))
collated.append(str(paymaya_bad_ps_final.print_topics()))

In [589]:
app_store = ['GCash Good AS', 'GCash Bad AS',
             'GCash Good PS', 'GCash Bad PS',
             'PayMaya Good AS', 'PayMaya Bad AS',
             'PayMaya Good PS', 'PayMaya Bad PS']

df_topics = pd.DataFrame(list(zip(app_store,collated)), columns = ['App Store','Topics'])

In [590]:
df_topics

,App Store,Topics
0,GCash Good AS,"[(0, '0.014*""money"" + 0.014*""convenient"" + 0.0..."
1,GCash Bad AS,"[(0, '0.014*""money"" + 0.010*""use"" + 0.009*""fix..."
2,GCash Good PS,"[(0, '0.201*""good"" + 0.128*""nice"" + 0.067*""gre..."
3,GCash Bad PS,"[(0, '0.023*""update"" + 0.020*""always"" + 0.015*..."
4,PayMaya Good AS,"[(0, '0.014*""use"" + 0.012*""convenient"" + 0.010..."
5,PayMaya Bad AS,"[(0, '0.015*""money"" + 0.012*""upgrade"" + 0.009*..."
6,PayMaya Good PS,"[(0, '0.114*""good"" + 0.082*""nice"" + 0.059*""gre..."
7,PayMaya Bad PS,"[(0, '0.015*""money"" + 0.015*""upgrade"" + 0.012*..."


In [591]:
date = datetime.now().strftime("%Y_%m_%d-%I-%M-%S_%p")

In [592]:
df_topics.to_csv(f"Results/Topics/Topics_{date}.csv", index= True)

In [ ]:
models = [gcash_good_as_final, gcash_bad_as_final,
          gcash_good_ps_final, gcash_bad_ps_final,
          paymaya_good_as_final, paymaya_bad_as_final,
          paymaya_good_ps_final, paymaya_bad_ps_final,]

In [ ]:
joblib.dump(models, f"Models/Model_{date}")

### IV. Topic Labeling
---